In [1]:
import pandas as pd
from sqlalchemy import create_engine
from geopy.geocoders import Nominatim
import numpy as np

In [2]:
#import csv
real_estate_file = 'data/nyc-rolling-sales.csv'
real_estate_df = pd.read_csv(real_estate_file)


real_estate_df

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,,...,5,0,5,1633,6440,1900,2,C2,6625000,7/19/17 0:00
1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,,...,28,3,31,4616,18690,1900,2,C7,-,12/14/16 0:00
2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,,...,16,1,17,2212,7803,1900,2,C7,-,12/9/16 0:00
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,,...,10,0,10,2272,6794,1913,2,C4,3936272,9/23/16 0:00
4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,,...,6,0,6,2369,4615,1900,2,C2,8000000,11/17/16 0:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84543,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,34,,B9,37 QUAIL LANE,,...,2,0,2,2400,2575,1998,1,B9,450000,11/28/16 0:00
84544,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,78,,B9,32 PHEASANT LANE,,...,2,0,2,2498,2377,1998,1,B9,550000,4/21/17 0:00
84545,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7351,60,,B2,49 PITNEY AVENUE,,...,2,0,2,4000,1496,1925,1,B2,460000,7/5/17 0:00
84546,5,WOODROW,22 STORE BUILDINGS,4,7100,28,,K6,2730 ARTHUR KILL ROAD,,...,0,7,7,208033,64117,2001,4,K6,11693337,12/21/16 0:00


In [3]:
# change address column type
real_estate_df = real_estate_df.astype({"ADDRESS": object})

real_estate_df.dtypes

BOROUGH                            int64
NEIGHBORHOOD                      object
BUILDING CLASS CATEGORY           object
TAX CLASS AT PRESENT              object
BLOCK                              int64
LOT                                int64
EASE-MENT                         object
BUILDING CLASS AT PRESENT         object
ADDRESS                           object
APARTMENT NUMBER                  object
ZIP CODE                           int64
RESIDENTIAL UNITS                  int64
COMMERCIAL UNITS                   int64
TOTAL UNITS                        int64
LAND SQUARE FEET                  object
GROSS SQUARE FEET                 object
YEAR BUILT                         int64
TAX CLASS AT TIME OF SALE          int64
BUILDING CLASS AT TIME OF SALE    object
SALE PRICE                        object
SALE DATE                         object
dtype: object

In [4]:
#change price column type

real_estate_df['SALE PRICE'] = real_estate_df['SALE PRICE'].fillna(0)

# real_estate_df['SALE PRICE'].str.replace('-','0')

real_estate_df['SALE PRICE'] = real_estate_df['SALE PRICE'].str.replace('-', "0")

real_estate_df['SALE PRICE'] = real_estate_df['SALE PRICE'].astype(str).astype(int)

real_estate_df.sample(50)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
68754,4,QUEENS VILLAGE,01 ONE FAMILY DWELLINGS,1,10666,98,,A1,89-92 221ST STREET,,...,1,0,1,3000,1520,1925,1,A1,499000,7/11/17 0:00
71289,4,ROSEDALE,01 ONE FAMILY DWELLINGS,1,13207,57,,A2,133-68 244TH STREET,,...,1,0,1,4000,1094,1950,1,A2,0,10/17/16 0:00
3061,1,GRAMERCY,02 TWO FAMILY DWELLINGS,1,899,38,,B9,236 EAST 19TH STREET,,...,2,0,2,1750,3880,1901,1,B9,7350000,11/9/16 0:00
35676,3,DOWNTOWN-FULTON MALL,22 STORE BUILDINGS,4,149,15,,K4,585 FULTON STREET,,...,0,5,5,1837,5511,1915,4,K4,0,3/29/17 0:00
44983,3,PARK SLOPE,15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,955,1002,,R1,593 PRESIDENT STREET,1R,...,1,0,1,0,0,1910,2,R1,100,6/5/17 0:00
14619,1,UPPER EAST SIDE (79-96),13 CONDOS - ELEVATOR APARTMENTS,2,1539,1425,,R4,200 EAST 94TH STREET,714,...,1,0,1,-,-,1986,2,R4,1633577,11/28/16 0:00
63347,4,HOWARD BEACH,09 COOPS - WALKUP APARTMENTS,2,11451,50,,C6,"90-17 SHORE PARKWAY, 83",,...,0,0,0,-,-,1958,2,C6,160000,7/11/17 0:00
59412,4,FLUSHING-SOUTH,01 ONE FAMILY DWELLINGS,1,6982,51,,A5,74-14 175 STREET,,...,1,0,1,2000,1980,1940,1,A5,698888,1/12/17 0:00
53173,4,COLLEGE POINT,02 TWO FAMILY DWELLINGS,1,4073,29,,S2,119-20 15TH AVENUE,,...,2,1,3,2800,3040,1930,1,S2,0,3/13/17 0:00
49776,4,ASTORIA,02 TWO FAMILY DWELLINGS,1,633,6,,B2,25-77 35TH STREET,,...,2,0,2,2225,1332,1920,1,B2,0,1/9/17 0:00


In [5]:
real_estate_df.dtypes

BOROUGH                            int64
NEIGHBORHOOD                      object
BUILDING CLASS CATEGORY           object
TAX CLASS AT PRESENT              object
BLOCK                              int64
LOT                                int64
EASE-MENT                         object
BUILDING CLASS AT PRESENT         object
ADDRESS                           object
APARTMENT NUMBER                  object
ZIP CODE                           int64
RESIDENTIAL UNITS                  int64
COMMERCIAL UNITS                   int64
TOTAL UNITS                        int64
LAND SQUARE FEET                  object
GROSS SQUARE FEET                 object
YEAR BUILT                         int64
TAX CLASS AT TIME OF SALE          int64
BUILDING CLASS AT TIME OF SALE    object
SALE PRICE                         int64
SALE DATE                         object
dtype: object

In [6]:
#add borough depending on number definfed by the data documentation.
borough_name = []


for b in real_estate_df['BOROUGH']:
    if  b == 1:
        borough_name.append("Manhattan")
        
    elif b == 2:
        borough_name.append("Bronx") 
        
    elif b == 3: 
        borough_name.append("Brooklyn") 
        
    elif b == 4:
        borough_name.append("Queens")
        
    else:
        borough_name.append("Staten Island") 
        

real_estate_df["borough_name"] = borough_name
real_estate_df.sample(50)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,borough_name
76687,5,ARROCHAR-SHORE ACRES,13 CONDOS - ELEVATOR APARTMENTS,2,2832,1017,,R4,6 NEW LANE,2D,...,0,1,-,-,1987,2,R4,0,1/2/17 0:00,Staten Island
21783,2,MOTT HAVEN/PORT MORRIS,41 TAX CLASS 4 - OTHER,4,2278,59,,Z9,109 BRUCKNER BOULEVARD,,...,1,1,4700,4418,1998,4,Z9,0,5/23/17 0:00,Bronx
72870,4,SOUTH OZONE PARK,01 ONE FAMILY DWELLINGS,1,11627,7,,A1,122-14 111TH AVENUE,,...,0,1,2000,848,1930,1,A1,439000,10/31/16 0:00,Queens
37994,3,FLATBUSH-EAST,01 ONE FAMILY DWELLINGS,1,7745,12,,A9,1755 ALBANY AVENUE,,...,0,1,2000,1526,1930,1,A9,345774,2/8/17 0:00,Brooklyn
37571,3,FLATBUSH-CENTRAL,10 COOPS - ELEVATOR APARTMENTS,2,5217,18,,D4,"616 EAST 18TH STREET, 4E",,...,0,0,0,0,1961,2,D4,765942,7/24/17 0:00,Brooklyn
48096,3,WILLIAMSBURG-EAST,14 RENTALS - 4-10 UNIT,2B,3097,26,,S5,109 MOORE STREET,,...,1,7,1875,7406,2004,2,S5,1000000,5/10/17 0:00,Brooklyn
49782,4,ASTORIA,02 TWO FAMILY DWELLINGS,1,651,62,,B1,30-44 37TH STREET,,...,0,2,2462,2720,1925,1,B1,1365000,12/5/16 0:00,Queens
29951,3,BOROUGH PARK,15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,5433,1607,,R1,4503 15TH AVENUE,4A,...,0,1,0,0,2011,2,R1,10,11/14/16 0:00,Brooklyn
57553,4,FLUSHING-NORTH,10 COOPS - ELEVATOR APARTMENTS,2,5024,24,,D4,"144-50 38TH AVENUE, 3B",,...,0,0,-,-,1950,2,D4,240000,4/5/17 0:00,Queens
65819,4,LAURELTON,02 TWO FAMILY DWELLINGS,1,13174,64,,B3,137-50 233RD STREET,,...,0,2,3142,1428,1920,1,B3,0,9/3/16 0:00,Queens


In [7]:
#get rid of appt numbers
real_estate_df['ADDRESS'] = real_estate_df['ADDRESS'].str.split(",", n = 1, expand = True)


real_estate_df.sample(50)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,borough_name
59314,4,FLUSHING-SOUTH,01 ONE FAMILY DWELLINGS,1,6844,42,,A1,73-06 164TH STREET,,...,0,1,3822,1386,1920,1,A1,0,4/21/17 0:00,Queens
49772,4,ASTORIA,02 TWO FAMILY DWELLINGS,1,626,52,,B1,30-18 35 STREET,,...,0,2,2600,2400,1930,1,B1,0,11/22/16 0:00,Queens
67246,4,MIDDLE VILLAGE,01 ONE FAMILY DWELLINGS,1,3004,23,,A5,64-36 79TH STREET,,...,0,1,1800,1260,1925,1,A5,0,7/20/17 0:00,Queens
37865,3,FLATBUSH-EAST,01 ONE FAMILY DWELLINGS,1,4986,77,,A5,775 E. 39 STREET,,...,0,1,1600,1520,1940,1,A5,0,6/14/17 0:00,Brooklyn
79758,5,MARINERS HARBOR,27 FACTORIES,4,1257,75,,F1,234 GRANDVIEW AVENUE,,...,1,1,10300,5000,1950,4,F1,0,11/7/16 0:00,Staten Island
32106,3,BUSHWICK,29 COMMERCIAL GARAGES,4,3139,36,,G7,11 MONTIETH STREET,,...,0,0,1875,0,0,4,G7,0,12/30/16 0:00,Brooklyn
47642,3,WILLIAMSBURG-CENTRAL,13 CONDOS - ELEVATOR APARTMENTS,2,2240,2110,,R4,40 MIDDLETON STREET,5B,...,0,1,0,0,2002,2,R4,0,12/1/16 0:00,Brooklyn
64007,4,JACKSON HEIGHTS,10 COOPS - ELEVATOR APARTMENTS,2,1254,12,,D4,33-15 81ST STREET,,...,0,0,-,-,1951,2,D4,245000,3/16/17 0:00,Queens
84420,5,WOODROW,01 ONE FAMILY DWELLINGS,1,6919,105,,A3,1009 IONIA AVENUE,,...,0,1,10000,4326,1985,1,A3,0,4/27/17 0:00,Staten Island
67691,4,OAKLAND GARDENS,01 ONE FAMILY DWELLINGS,1,7761,95,,A1,217-48 77TH AVENUE,,...,0,1,6240,2056,1940,1,A1,950000,7/12/17 0:00,Queens


In [11]:
#making new column with full address

real_estate_df["Full_Address"] = real_estate_df["ADDRESS"] + " "  + real_estate_df["borough_name"]

real_estate_df

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,borough_name,Full_Address
0,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,,...,5,1633,6440,1900,2,C2,6625000,7/19/17 0:00,Manhattan,153 AVENUE B Manhattan
1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,,...,31,4616,18690,1900,2,C7,0,12/14/16 0:00,Manhattan,234 EAST 4TH STREET Manhattan
2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,,...,17,2212,7803,1900,2,C7,0,12/9/16 0:00,Manhattan,197 EAST 3RD STREET Manhattan
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,,...,10,2272,6794,1913,2,C4,3936272,9/23/16 0:00,Manhattan,154 EAST 7TH STREET Manhattan
4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,,...,6,2369,4615,1900,2,C2,8000000,11/17/16 0:00,Manhattan,301 EAST 10TH STREET Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84543,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,34,,B9,37 QUAIL LANE,,...,2,2400,2575,1998,1,B9,450000,11/28/16 0:00,Staten Island,37 QUAIL LANE Staten Island
84544,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,78,,B9,32 PHEASANT LANE,,...,2,2498,2377,1998,1,B9,550000,4/21/17 0:00,Staten Island,32 PHEASANT LANE Staten Island
84545,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7351,60,,B2,49 PITNEY AVENUE,,...,2,4000,1496,1925,1,B2,460000,7/5/17 0:00,Staten Island,49 PITNEY AVENUE Staten Island
84546,5,WOODROW,22 STORE BUILDINGS,4,7100,28,,K6,2730 ARTHUR KILL ROAD,,...,7,208033,64117,2001,4,K6,11693337,12/21/16 0:00,Staten Island,2730 ARTHUR KILL ROAD Staten Island


In [19]:
#picking columns and assigning to new dataframe
real_estate_cols = ['Full_Address','NEIGHBORHOOD', 'TAX CLASS AT PRESENT',
                   'ZIP CODE', 'SALE PRICE', 'BUILDING CLASS AT TIME OF SALE', 'borough_name']

real_estate_clean_df = real_estate_df[real_estate_cols]

real_estate_clean_df

,Full_Address,NEIGHBORHOOD,TAX CLASS AT PRESENT,ZIP CODE,SALE PRICE,BUILDING CLASS AT TIME OF SALE,borough_name
0,153 AVENUE B Manhattan,ALPHABET CITY,2A,10009,6625000,C2,Manhattan
1,234 EAST 4TH STREET Manhattan,ALPHABET CITY,2,10009,0,C7,Manhattan
2,197 EAST 3RD STREET Manhattan,ALPHABET CITY,2,10009,0,C7,Manhattan
3,154 EAST 7TH STREET Manhattan,ALPHABET CITY,2B,10009,3936272,C4,Manhattan
4,301 EAST 10TH STREET Manhattan,ALPHABET CITY,2A,10009,8000000,C2,Manhattan
...,...,...,...,...,...,...,...
84543,37 QUAIL LANE Staten Island,WOODROW,1,10309,450000,B9,Staten Island
84544,32 PHEASANT LANE Staten Island,WOODROW,1,10309,550000,B9,Staten Island
84545,49 PITNEY AVENUE Staten Island,WOODROW,1,10309,460000,B2,Staten Island
84546,2730 ARTHUR KILL ROAD Staten Island,WOODROW,4,10309,11693337,K6,Staten Island


In [20]:
real_estate_clean_df.apply(lambda col: col.drop_duplicates().reset_index(drop=True))


,Full_Address,NEIGHBORHOOD,TAX CLASS AT PRESENT,ZIP CODE,SALE PRICE,BUILDING CLASS AT TIME OF SALE,borough_name
0,153 AVENUE B Manhattan,ALPHABET CITY,2A,10009.0,6625000.0,C2,Manhattan
1,234 EAST 4TH STREET Manhattan,CHELSEA,2,0.0,0.0,C7,Bronx
2,197 EAST 3RD STREET Manhattan,CHINATOWN,2B,10002.0,3936272.0,C4,Brooklyn
3,154 EAST 7TH STREET Manhattan,CIVIC CENTER,2C,10011.0,8000000.0,D5,Queens
4,301 EAST 10TH STREET Manhattan,CLINTON,,10001.0,3192840.0,D9,Staten Island
...,...,...,...,...,...,...,...
58586,37 QUAIL LANE Staten Island,NaN,NaN,NaN,NaN,NaN,NaN
58587,32 PHEASANT LANE Staten Island,NaN,NaN,NaN,NaN,NaN,NaN
58588,49 PITNEY AVENUE Staten Island,NaN,NaN,NaN,NaN,NaN,NaN
58589,2730 ARTHUR KILL ROAD Staten Island,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# new_deduped_df = deduped_df.groupby("Full_Address", sort=False)



NameError: name 'deduped_df' is not defined

In [ ]:
deduped_df

In [ ]:
#create lat and long columns 

geolocator = Nominatim(user_agent="user_agent")
location = geolocator.geocode("327 EAST 3rd STREET Manhattan")

print((location.latitude, location.longitude))

In [ ]:


lat = []
long = []

i = 0

geolocator = Nominatim(user_agent="user_agent")

for address in real_estate_clean_df["Full_Address"]:
    
    addr = geolocator.geocode(address, timeout= 1)  
    if addr == None:
        lat.append(np.nan)
        long.append(np.nan)
        i += 1
        continue 
        
    else:
        i += 1
        lat.append(addr.latitude)
        long.append(addr.longitude)
        
    
    if i%500 == 0:
        print(addr.latitude, addr.longitude)
        
    else:
        pass
    
    
#     try:
#         i += 1
#         lat.append(addr.latitude)
#         long.append(addr.longitude)
#     except:
#         i += 1
#         print("none")
#         lat.append(np.nan)
#         long.append(np.nan)
#         pass
    
    
    
#     if addr is None:
#         lat.append(None)
#         long.append(None)
        
#     else:
#         latitude = addr.latitude
#         longitude = addr.longitude  
    
#         lat.append(latitude)
#         long.append(longitude)

real_estate_clean_df['latitude'] = lat
real_estate_clean_df['longitude'] = long

In [ ]:
real_estate_clean_df[real_estate_clean_df['Full_Address'].str.contains('10 WEST 15')]